# 02 - Data from the Web

## Deadline
Wednesday October 25, 2017 at 11:59PM

## Important Notes
* Make sure you push on GitHub your Notebook with all the cells already evaluated (i.e., you don't want your colleagues to generate unnecessary Web traffic during the peer review)
* Don't forget to add a textual description of your thought process, the assumptions you made, and the solution you plan to implement!
* Please write all your comments in English, and use meaningful variable names in your code.

## Background
In this homework we will extract interesting information from www.topuniversities.com and www.timeshighereducation.com, two platforms that maintain a global ranking of worldwide universities. This ranking is not offered as a downloadable dataset, so you will have to find a way to scrape the information we need!
You are not allowed to download manually the entire ranking -- rather you have to understand how the server loads it in your browser. For this task, Postman with the Interceptor extension can help you greatly. We recommend that you watch this [brief tutorial](https://www.youtube.com/watch?v=jBjXVrS8nXs&list=PLM-7VG-sgbtD8qBnGeQM5nvlpqB_ktaLZ&autoplay=1) to understand quickly how to use it.

## Assignment
1. Obtain the 200 top-ranking universities in www.topuniversities.com ([ranking 2018](https://www.topuniversities.com/university-rankings/world-university-rankings/2018)). In particular, extract the following fields for each university: name, rank, country and region, number of faculty members (international and total) and number of students (international and total). Some information is not available in the main list and you have to find them in the [details page](https://www.topuniversities.com/universities/ecole-polytechnique-fédérale-de-lausanne-epfl).
Store the resulting dataset in a pandas DataFrame and answer the following questions:
    - Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?
    - Answer the previous question aggregating the data by (c) country and (d) region.

Plot your data using bar charts and describe briefly what you observed.

2. Obtain the 200 top-ranking universities in www.timeshighereducation.com ([ranking 2018](http://timeshighereducation.com/world-university-rankings/2018/world-ranking)). Repeat the analysis of the previous point and discuss briefly what you observed.

3. Merge the two DataFrames created in questions 1 and 2 using university names. Match universities' names as well as you can, and explain your strategy. Keep track of the original position in both rankings.

4. Find useful insights in the data by performing an exploratory analysis. Can you find a strong correlation between any pair of variables in the dataset you just created? Example: when a university is strong in its international dimension, can you observe a consistency both for students and faculty members?

5. Can you find the best university taking in consideration both rankings? Explain your approach.

Hints:
- Keep your Notebook clean and don't print the verbose output of the requests if this does not add useful information for the reader.
- In case of tie, use the order defined in the webpage.


In [1]:
import math
import re
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from functools import reduce
from tqdm import *
import matplotlib.pyplot as plt

#https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/100/sort_by/rank/sort_order/asc/cols/stats

# Part 1 : Scraping www.topuniversities.com

In [2]:
url = "https://www.topuniversities.com"

In [3]:
def detail_per_uni(urls) :
    
    soup = BeautifulSoup(urls.text,"html.parser")
    
    total_faculty  = soup.find('div',class_='total faculty')
    
    tf = 0
    ts = 0 
    ti = 0 
    
    if(total_faculty != None) :
        total_faculty = total_faculty.find('div',class_='number').text
        total_student = soup.find('div',class_='total student').find('div',class_='number').text
        total_international = soup.find('div',class_ = 'total inter').find('div',class_='number').text
        tf = int(total_faculty[1:-1].replace(',',''))
        ts = int(total_student[1:-1].replace(',',''))
        ti = int(total_international[1:-1].replace(',',''))
    else :
        #if the data isn't availble on the site
        tf = 0
        ts = 0
        ti = 0
    return {'Total faculty members': tf, 'Total students':ts,'International students':ti}

def scrape_uni_detail(urls):
    parsed_url = []
    
    for i,url in enumerate(tqdm_notebook(urls)) : 
        parsed_url.append(detail_per_uni(url))
        #print("progress ",i)

    return parsed_url

In [5]:
def noequals(x) :
    """
    checks if a certain string begins with '='
    """
    return None==re.match('(=)\w+', x) 

In [7]:
r = requests.get('https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508105606999')

data= dict(json.loads(r.text))

data= pd.DataFrame(data['data'])[['title','rank_display','region','country','url']]

data.columns = ['University','Ranking', 'Region','Country','URL']
data.Ranking = data.Ranking.apply(lambda r : str(r))

rankings = list(data.Ranking)
#we clean the ranking column so that we don't have equals
rankings =  pd.Series([r if noequals(r) else r[1:] for r in rankings])

data['Ranking'] = rankings



In [8]:
urls = []

for i in tqdm_notebook(range(100)): 
    urls.append(requests.get(url+data.URL[i]))

HBox(children=(IntProgress(value=0), HTML(value='')))

In [9]:
for i in tqdm_notebook(range(100,200)):
    urls.append(requests.get(url+data.URL[i]))


HBox(children=(IntProgress(value=0), HTML(value='')))

In [10]:
details_uni = scrape_uni_detail(urls)

HBox(children=(IntProgress(value=0, max=200), HTML(value='')))

In [11]:
details_uni = pd.DataFrame(details_uni)

d = data.copy()
d = d.loc[:199]
d = d.drop('URL',axis=1)

final_data = pd.concat([d,details_uni],axis=1)
#final_data =final_data.set_index('University')

#### Sanity check for unique index

In [12]:
final_data.index.is_unique

True

In [13]:
final_data

,University,Ranking,Region,Country,International students,Total faculty members,Total students
0,Massachusetts Institute of Technology (MIT),1,North America,United States,3717,2982,11067
1,Stanford University,2,North America,United States,3611,4285,15878
2,Harvard University,3,North America,United States,5266,4350,22429
3,California Institute of Technology (Caltech),4,North America,United States,647,953,2255
4,University of Cambridge,5,Europe,United Kingdom,6699,5490,18770
5,University of Oxford,6,Europe,United Kingdom,7353,6750,19720
6,UCL (University College London),7,Europe,United Kingdom,14854,6345,31080
7,Imperial College London,8,Europe,United Kingdom,8746,3930,16090
8,University of Chicago,9,North America,United States,3379,2449,13557
9,ETH Zurich - Swiss Federal Institute of Techno...,10,Europe,Switzerland,7563,2477,19815


## Ranking aggregation
We can now answer the following questions:
- Which are the best universities in term of: 
    * (a) ratio between faculty members and students
    * (b) ratio of international students
- Answer the previous question aggregating the data by 
    * (c) country
    * (d) region

In [14]:
final_data['Ratio faculty members and students'] = final_data['Total faculty members']/final_data['Total students']
final_data['Ratio faculty members and international students'] = final_data['Total faculty members']/final_data['International students']

ratio_faculty_intern_max = final_data['Ratio faculty members and international students'].idxmax()
ratio_faculty_total_students = final_data['Ratio faculty members and students'].idxmax()
#final_data.loc[final_data['Ratio faculty members and international students'] == number]

In [15]:
(pd.DataFrame(final_data.loc[ratio_faculty_intern_max]).T)

,University,Ranking,Region,Country,International students,Total faculty members,Total students,Ratio faculty members and students,Ratio faculty members and international students
179,Indian Institute of Technology Bombay (IITB),179,Asia,India,93,876,9402,0.0931717,9.41935


In [16]:
pd.DataFrame(final_data.loc[ratio_faculty_total_students]).T

,University,Ranking,Region,Country,International students,Total faculty members,Total students,Ratio faculty members and students,Ratio faculty members and international students
3,California Institute of Technology (Caltech),4,North America,United States,647,953,2255,0.422616,1.47295


## Aggregated by country
Now, let's plot the same results aggregated by country.

In [17]:
# we just perfom a simply groupby action to do the aggregrate

def get_aggregate_data(column,final_data,groupby) :

    final_data_grouped_region = final_data.sort_values([column],ascending=False).groupby([groupby])

    dataframe = []
    for region,df in (final_data_grouped_region) :
        dataframe.append(pd.DataFrame(df.iloc[0]).T)

    final_data_grouped_region = (reduce(lambda a,b : pd.concat([a,b],axis=0),dataframe ))

    return final_data_grouped_region



### Ranking of countries in term of ratio of students to staff in their universities

In [18]:
get_aggregate_data('Ratio faculty members and students',final_data,'Country').reset_index().set_index('Country')

,index,University,Ranking,Region,International students,Total faculty members,Total students,Ratio faculty members and students,Ratio faculty members and international students
Country,,,,,,,,,
Argentina,74,Universidad de Buenos Aires (UBA),75,Latin America,27109,16421,122301,0.134267,0.60574
Australia,19,The Australian National University,20,Oceania,5551,1600,14442,0.110788,0.288236
Austria,153,University of Vienna,154,Europe,14468,3411,45967,0.0742054,0.235762
Belgium,181,Vrije Universiteit Brussel (VUB),182,Europe,1853,1792,9284,0.19302,0.96708
Brazil,120,Universidade de São Paulo,121,Latin America,2086,5582,65711,0.0849477,2.67593
Canada,139,McMaster University,140,North America,3548,3231,23702,0.136318,0.910654
Chile,138,Pontificia Universidad Católica de Chile (UC),137,Latin America,991,2260,27003,0.0836944,2.28052
China,24,Tsinghua University,25,Asia,4072,5506,36300,0.15168,1.35216
Denmark,116,Technical University of Denmark,116,Europe,2098,2117,8878,0.238455,1.00906


### Ranking of countries in term of ratio of international students in their universities

In [19]:
get_aggregate_data('Ratio faculty members and international students',final_data,'Country').reset_index().set_index('Country')

,index,University,Ranking,Region,International students,Total faculty members,Total students,Ratio faculty members and students,Ratio faculty members and international students
Country,,,,,,,,,
Argentina,74,Universidad de Buenos Aires (UBA),75,Latin America,27109,16421,122301,0.134267,0.60574
Australia,93,The University of Western Australia,93,Oceania,4575,1420,18531,0.0766284,0.310383
Austria,153,University of Vienna,154,Europe,14468,3411,45967,0.0742054,0.235762
Belgium,181,Vrije Universiteit Brussel (VUB),182,Europe,1853,1792,9284,0.19302,0.96708
Brazil,120,Universidade de São Paulo,121,Latin America,2086,5582,65711,0.0849477,2.67593
Canada,139,McMaster University,140,North America,3548,3231,23702,0.136318,0.910654
Chile,138,Pontificia Universidad Católica de Chile (UC),137,Latin America,991,2260,27003,0.0836944,2.28052
China,96,University of Science and Technology of China,97,Asia,548,1992,16032,0.124251,3.63504
Denmark,73,University of Copenhagen,73,Europe,3683,7483,32119,0.232977,2.03177


## Aggregated by region
Now, let's plot the same results aggregated by region.

### Ranking of regions in term of ratio of international students in their universities

In [ ]:
get_aggregate_data('Ratio faculty members and international students',final_data,'Region').set_index('Region')

### Ranking of regions in term of ratio of students to staff in their universities

In [ ]:
get_aggregate_data('Ratio faculty members and students',final_data,'Region').set_index('Region')

# Part 2 : Scraping www.timeshighereducation.com
Assumptions
- We are using [this site](https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/-1/sort_by/rank/sort_order/asc/cols/stats) which contains the rankings of 2018.
- Rank saved within `<td class="rank ..."><td>` element
- University saved within `<a href="..." class="ranking-institution-title ..."></a>` element
- Country saved within `<div class="location"></div>` element

First, the the data returned in the json file returned in the `GET` request are imported and Universities that are not within the top 200 are removed

In [ ]:
def loadData(URL):
    # make a request
    r = requests.get(URL)
    r = r.json()
    # parse HTML with BeautifulSoup
    df=pd.DataFrame(r['data'])
    return df
    
URL = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
df=loadData(URL)
print(df.columns)
display(df.head())

We do not need all columns, thus we extract only the ones relevant to the data we are looking for: _name, rank, country and region, number of faculty members (international and total) and number of students (international and total)._

These columns are derived as follows:
- `name`: column `name`
- `country`: `location`
- `region`: in detail URL?
- `number of faculty members (international)`: ?
- `number of faculty members (total)`: `stats_number_students` times `stats_student_staff_ratio`
- `number of students (international)`: `stats_number_students` times `stats_pc_intl_students`
- `number of students (total)`: `stats_number_students`

We first have to do some data cleanup:
- remove equal sign (`=`) in field `rank` (Universities having the same rank) and removing universities having a rank above 200
-  convert percentage of students in field `stats_pc_intl_students` (e.g. 83%) to float (e.g., 0.83) for multiplication
- Remove commas and cast to  type `int` for column `stats_number_students` ("number of students")

In [ ]:
df = loadData(URL)
columns = ['name','rank','location','stats_number_students','stats_student_staff_ratio',
           'stats_pc_intl_students','nid']
columns_new_names = ['name','rank','country','n_students','ratio_student_staff','pc_students_int','nid']

df = df[columns] # selections the columns
df.columns = (columns_new_names) # renaming the columns

# Cleaning the rank attribute
df['rank'] = df['rank'].str.replace('=','')
df = df[np.isin(df['rank'], ['1001+','201–250','251–300','301–350','351–400','401–500','501–600','601–800','801–1000'],
                invert=True)]
df['rank'] = df['rank'].astype(int)

# Convert percentage (e.g. 83%) to float (e.g., 0.83) for multiplication
def divide(x, n = 100):
    return float(x)/n

df['pc_students_int'] = df['pc_students_int'].str.replace('%','').apply(divide, n=100)

df['ratio_student_staff'] = df['ratio_student_staff'].str.replace(',', '').astype(float)

# clean up: stats on number of students
df['n_students'] = df['n_students'].str.replace(',', '').astype(int)
df.head()

We can now derive new columns for the missing data fields based on the existing columns.

In [ ]:
# derived columns
df['n_staff'] = np.round(df['n_students'] / (df['ratio_student_staff'])).astype(int)
df['n_students_int'] = np.round(df['n_students'] * df['pc_students_int']).astype(int)
#df['ratio_staff_student']=df['ratio_student_staff']**-1
display(df.head())

## Ranking aggregation
We can now answer the following questions:
- Which are the best universities in term of: 
    * (a) ratio between faculty members and students
    * (b) ratio of international students
- Answer the previous question aggregating the data by 
    * (c) country
    * (d) region

Plot your data using bar charts and describe briefly what you observed.

In [ ]:
def plot_histogram(df, x, y, title, xlab, figsize=(20,10)):
    """
    This functions plots a histogram adding the values on top of the bins.
    
    df: dataframe
    x: column name for x axis
    y: column name for y axis
    title: title to give to the histogram
    xlab: xlabel
    """
    x=df[x].values
    y=np.round(df[y],2)
    fig, ax = plt.subplots(figsize=figsize)
    width = 0.5 # the width of the bars
    ind = np.arange(len(y))  # the x locations for the groups
    ax.bar(ind, y, width, color="red")
    ax.set_xticks(ind)
    ax.set_xticklabels(x, rotation=90)
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(title)
    for i, v in enumerate(y):
        
        ax.text(i-width/2, v*1.02, str(v), color='red')

    plt.show()

### Ranking in term of ratio between students and faculty members

In [ ]:
# ratio between faculty members and students
df_ratio_ss = df.sort_values(by=['ratio_student_staff']).head(10)
plot_histogram(df_ratio_ss,"name","ratio_student_staff", title='Ratio students to staff',
               xlab='University', figsize=(7,5))

### Ranking in term of ratio of international students

In [ ]:
# ratio of international students
df_ratio_is = df.sort_values(ascending=False,by=['pc_students_int']).head(10)
plot_histogram(df_ratio_is,'name','pc_students_int', title='International Students Percentage',
               xlab='University', figsize=(7,5))

## Aggregated: by country
Now, let's plot the same results aggregated by country.

### Ranking of countries in term of ratio of students to staff in their universities

In [ ]:
df_c_ratio_ss=df.groupby("country").mean().sort_values(ascending=False,by=['ratio_student_staff']).head(10)
df_c_ratio_ss.reset_index(inplace=True)

In [ ]:
plot_histogram(df_c_ratio_ss,"country","ratio_student_staff", title='Ratio students to staff',
               xlab='University', figsize=(7,5))

### Ranking of countries in term of international students percentage in their universities

In [ ]:
df_c_ratio_is=df.groupby("country").mean().sort_values(ascending=False,by=['pc_students_int']).head(10)
df_c_ratio_is.reset_index(inplace=True)

In [ ]:
plot_histogram(df_c_ratio_is,"country","pc_students_int", title='International Students Percentage',
               xlab='University', figsize=(7,5))